## LandCover Demo - Using the ArcGIS Python API
This notebook reviews the workflow for extracting land cover data for a spatially selected watershed.

### Pseudocode
* Get the user point, as lat/long coordinates (Alternatively, geocode address)
* Identify the HUC12 in which it is located
 * Search AGOL for a watershed boundary datatset
 * Retrive the HUC12 layer from the results
 * Query the HUC12 intersecting the user point and report its HUC8 value
* Extract land cover within the selected HUC
 * Browse Living Atlas for an appropriate land cover dataset (NLCD)
 * Retrieve the land cover as an imagery service
 * Filter the images for a specific time
 * Extract the histogram for the pixels within the geometry

In [ ]:
#Import arcgis modules
from arcgis import GIS
from arcgis.geocoding import geocode
from arcgis.features import FeatureLayer, GeoAccessor
from arcgis.geometry import Point, filters

#Import pandas
import pandas as pd

#Connect to AGOL
gis = GIS('home')

Construct a Point object from the user coordinates
 * References: 
  * https://developers.arcgis.com/python/api-reference/arcgis.geometry.html#
  * https://developers.arcgis.com/python/api-reference/arcgis.geometry.html#point

In [ ]:
# Set user coordinates
userLat = 36.0010
userLng = -78.9991

In [ ]:
#Create the point object
the_pt = Point({
    "x" : userLng, 
    "y" : userLat, 
    "spatialReference" : {"wkid" : 4326}
})

In [ ]:
#Alternatively, use geocoding
nsoe = geocode('Nicholas School of the Environment',as_featureset=True).sdf
#Get the geometry of the first item returned
the_point = nsoe.loc[0,'SHAPE']
#Print the point
print(the_point)

### Task 1. Finding Watershed
The watershed boundary dataset is served via ESRI's Living Atlas. 

In [ ]:
#Search for a dataset of HUCs 
search_results = gis.content.search(
    query='"Watershed Boundary Dataset" owner:esri_environment',
    item_type='Feature',
    outside_org=True)
search_results

In [ ]:
#Extract the 1st item (the HUC12s) and view it
huc_item = search_results[0]
huc_item

In [ ]:
#Fetch its Item ID
theID = huc_item.id
theID

In [ ]:
#For future reference, retrieve Watershed from its item ID
item = gis.content.get('b60aa1d756b245cf9db03a92254af878')

In [ ]:
#Reveal the layers, printing their info
layers = item.layers
for layer in layers:
    print("Name:",layer.properties.name)
    print("Type:", layer.properties.type)

Layers: https://developers.arcgis.com/python/guide/working-with-feature-layers-and-features/

In [ ]:
#Extract the one and only layer 
huc_layer = layers[0]

#Show it's URL
print(huc_layer.url)

The layer's [URL](https://services.arcgis.com/P3ePLMYs2RVChkJx/arcgis/rest/services/Watershed_Boundary_Dataset_HUC_8s/FeatureServer/0) is its "REST Service Endpoint" and it includes useful information on how to work with the layer. 

We can also get the feature layer directly from its URL using the [`FeatureLayer`](https://developers.arcgis.com/python/latest/api-reference/arcgis.features.toc.html#featurelayer) object.

In [ ]:
#Fetch a feature layer from its url
huc_layer = FeatureLayer(url='https://services.arcgis.com/P3ePLMYs2RVChkJx/arcgis/rest/services/Watershed_Boundary_Dataset_HUC_12s/FeatureServer/0')
type(huc_layer)

## Querying the feature layer
We could convert this layer into a Spatial Dataframe using the code `GeoAccessor.from_layer(huc8_layer)`, but as it's a large dataset, that would take some time. So instead, we'll [query](https://developers.arcgis.com/python/latest/api-reference/arcgis.features.toc.html#arcgis.features.FeatureLayer.query) the feature layer for features intersecting our point.

* To do this we need to create a [geometry filter](https://developers.arcgis.com/python/latest/api-reference/arcgis.geometry.filters.html#module-arcgis.geometry.filters) object, on specific for intersecting the point we created above.
* Then we'll run the query command on the feature layer, which will return a [FeatureSet](https://developers.arcgis.com/python/latest/api-reference/arcgis.features.toc.html#featureset) object.
* And finally, we'll convert our feature set to a Spatial Dataframe using the `sdf` command. 


In [ ]:
#Create a filter object from our point
from arcgis.geometry import filters
point_filter = filters.intersects(the_pt)

In [ ]:
#Spatially query the layer
huc_fset = huc_layer.query(
    geometry_filter=point_filter,
    #distance=5000
)
type(huc_fset)

In [ ]:
#Convert to a spatial dataframe
huc_sdf = huc_fset.sdf
huc_sdf

In [ ]:
#Display the data
m = huc_sdf.spatial.plot()
#m.basemap = 'oceans'
m.draw(the_pt)
#m

## Extract land cover within the selected HUC
Now that we have the HUC we want to analyze, the next step is to use it to extract land cover data falling within it. 


### Finding and retrieving land cover data
To do this we need to find a land cover dataset and read it into our coding environment. Browsing the Living Atlas, we find the USA NLCD Land Cover service that will suit our needs: 
* View the item page for [USA NLCD Land Cover](https://dukeuniv.maps.arcgis.com/home/item.html?id=3ccf118ed80748909eb85c6d262b426f) service.
* Note its item ID: `3ccf118ed80748909eb85c6d262b426f`; we'll use that to fetch the data
* Note that it's an [Imagery Layer](https://developers.arcgis.com/python/latest/api-reference/arcgis.raster.toc.html#imagerylayer), this behaves differently than a Feature Layer
* Note also that the service is a **Time Series** datsaet, including many **Time Extents**
* Despite having many Time Extents, the Imagery Layer service only contains one Layer: [USA_NLCD_Land_Cover](https://landscape10.arcgis.com/arcgis/rest/services/USA_NLCD_Land_Cover/ImageServer)

In short, this is no ordinary raster; rather its a time series raster from which we'll have to extract specific dates in order to get a raster object we can work with. 

In [ ]:
#Fetch the NLCD Service item via its id
nlcd_item = gis.content.get('3ccf118ed80748909eb85c6d262b426f')
nlcd_item

In [ ]:
#Get the one and only layer
nlcd_layer = nlcd_item.layers[0]
type(nlcd_layer)

► *Note: we can get the ImageryLayer directly via the `ImageryLayer` object and the URL. However, because this is a subscription only layer, we need to specify the gis object when we import it.*

In [ ]:
#Get the ImageryLayer directly via the ImageryLayer object and the url
from arcgis.raster import ImageryLayer
nlcd_layer = ImageryLayer(
    url='https://landscape10.arcgis.com/arcgis/rest/services/USA_NLCD_Land_Cover/ImageServer',
    gis=gis
)
type(nlcd_layer)

### Exploring the image service
As mentioned above, this imagery service includes many time extents. As such, it has an associated feature layer that lists the properties (including spatial extents) of each of the rasters included in the service. We can access that feature layer by querying the image service, 

* First we'll extract the feature set associated with this ImageryLayer object, done with the `query()` function. 
* Then we'll convert this feature set to a spatial dataframe (sdf) so we can examine its contents. 

*→The feature set associated with an imagery layer includes information and spatial extents of the individual images contained in the ImageryLayer service.*

In [ ]:
#Extract the feature set associated with the ImageryLayer
nlcd_featureSet = nlcd_layer.query()

In [ ]:
#Convert to a spatial dataframe
nlcd_sdf = nlcd_featureSet.sdf

#Display the first three rows of the dataframe
nlcd_sdf.head()

Browsing this dataframe, we see the criteria from which we can subset specific NLCD raster datasets from the image service. 

### Selecting images from the ImageryLayer for use in analysis
In the above dataframe, each row represents an image that we analyze, and each column represents an attribute we can use to subset the images used in our analysis. We can also subset rows by filtering for those that intersect a specified *geometry* or that include a specified *time* or *time extent*. 

These criteria can be applied in a number of analytical functions associated with [ImageryLayer objects](https://developers.arcgis.com/python/latest/api-reference/arcgis.raster.toc.html#arcgis.raster.ImageryLayer). For example: 
* [`identify()`](https://developers.arcgis.com/python/latest/api-reference/arcgis.raster.toc.html#arcgis.raster.ImageryLayer.identify)
* [`filter_by()`](https://developers.arcgis.com/python/latest/api-reference/arcgis.raster.toc.html#arcgis.raster.ImageryLayer.filter_by)
* [`compute_histograms()`](https://developers.arcgis.com/python/latest/api-reference/arcgis.raster.toc.html#arcgis.raster.ImageryLayer.compute_histograms)
* [`get_samples()`](https://developers.arcgis.com/python/latest/api-reference/arcgis.raster.toc.html#arcgis.raster.ImageryLayer.get_samples)


In [ ]:
#List unique dataset names in the imagery layer
list(nlcd_sdf['Name'].unique())

##### Using `filter_by` to create a subset of images
* To filter images using attributes in the feature set dataframe, we use the `where` parameter, supplying a valid "where clause"
* To filter images using a time/time extent, we use the `time` parameter, supplying datetime objects
* To filter images using location, we use the `geometry` parmameter, supplying arcgis geometry objects 

In [ ]:
#First, extract the geometry from the HUC result above
the_geom = huc_sdf['SHAPE'][0]

In [ ]:
#Query the imagery layer
selected_images = nlcd_layer.filter_by(
    where="Name = 'USA_NLCD_Land_Cover_2016_conus'",    #Filter for just USA_NLCD_Land_Cover_2016_conus
    #where = "Name LIKE 'USA_NLCD_Land_Cover_%_conus'",  #Filter for all records with the name 'USA_NLCD_Land_Cover_????_conus' (where ???? is a wildcard)
    #time=[datetime(2016,1,1), datetime(2016,12,31)],    #Filter for datasets collected in 2016
    geometry=filters.intersects(the_geom),              #Filter for datasets that intersect the geometry
    lock_rasters=True
)

#Show the selected images as a dataframe
selected_images_df = selected_images.query().sdf
selected_images_df

The [`compute_histograms()`](https://developers.arcgis.com/python/latest/api-reference/arcgis.raster.toc.html#arcgis.raster.ImageryLayer.compute_histograms) returns a list of histograms for all raster bands computed for the ImageryLayer from the given extent. It returns a dictionary from which we can extract a list of the # of pixels in each class.

In [ ]:
#Extract the values within the selected geometry
the_histogram = selected_images.compute_histograms(
    geometry=the_geom,
    pixel_size=30)
#the_histogram

In [ ]:
#Extract just the counts
the_counts = the_histogram['histograms'][0]['counts']

#Convert the counts to a dataframe
df_results = pd.DataFrame(the_counts,columns=['Count'])
df_results.loc[df_results['Count']>0]

Always good to check our work. We'll compare the area of the geomtery to the sum area of the pixels in the histogram result.

In [ ]:
#Project the watershed boundary to UTM 17 and compute area
the_geom_area = the_geom.project_as({'wkid':32617}).area 

#Compute the sum of the count column, multiply by the cell size (30 x 30)
the_pixel_area = df_results['Count'].sum(axis=0) * 900

#Compare the two: should be close to 1
the_geom_area / the_pixel_area

Now we can link NLCD class names to the values to make nice plots. 

I have created and posted a CSV file of NLCD class values and name on GitHub that we can read into a dataframe.

In [ ]:
#Fetch the NLCD land cover classes
df_classes = pd.read_csv(
    'https://raw.githubusercontent.com/ENV859/ArcGIS-PythonAPI/refs/heads/master/data/NLCD.csv',
     index_col=0)
df_classes.head(3)

In [ ]:
#Merge with the classes DataFrame
df_histo = pd.merge(df_classes, df_results, left_index=True, right_index=True)
df_histo.plot.bar(x='Class Name', y='Count');